In [ ]:
# Google Colab Setup / Google Colab セットアップ
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import sys
    from pathlib import Path
    
    proj_root = Path("/content/drive/MyDrive/image_processing_tutorial-main")
    if str(proj_root) not in sys.path:
        sys.path.insert(0, str(proj_root))
    
    print("Google Colab environment detected and configured!")
    print("Google Colab環境が検出され、設定されました！")
else:
    print("Local Jupyter environment detected")
    print("ローカルJupyter環境が検出されました")

# Image Filtering Tutorial / 画像フィルタリングチュートリアル

This notebook demonstrates fundamental image processing techniques including smoothing filters, edge detection, and feature detection.

このノートブックでは、平滑化フィルタ、エッジ検出、特徴検出を含む基本的な画像処理技術を実演します。

1. Lena Image Download / レナ画像のダウンロード
2. Smoothing Filters / 平滑化フィルタ
3. Edge Detection with Sobel Filters / ソーベルフィルタによるエッジ検出
4. Harris Corner Detection / ハリスコーナー検出
5. SIFT Feature Detection / SIFT特徴点検出

In [ ]:
# Import required libraries / 必要なライブラリをインポート
import numpy as np
import matplotlib.pyplot as plt
import cv2
import requests
from skimage import filters, feature, io
from scipy import ndimage
import os

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Lena Image Download / レナ画像のダウンロード

We'll download the famous Lena image, which is commonly used in image processing tutorials.

画像処理チュートリアルでよく使用される有名なレナ画像をダウンロードします。

In [ ]:
def download_lena_image():
    """
    Download Lena image from multiple reliable sources with fallback
    複数の信頼できるソースからレナ画像をダウンロード（フォールバック付き）
    """
    # Multiple URLs in order of preference / 優先順位順の複数URL
    urls = [
        "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg",
        "https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png",
    ]
    
    # Headers to avoid bot detection / ボット検出を回避するためのヘッダー
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    for i, url in enumerate(urls):
        try:
            print(f"Trying source {i+1}/{len(urls)}: {url.split('/')[-1]}")
            print(f"ソース {i+1}/{len(urls)} を試行中: {url.split('/')[-1]}")
            
            response = requests.get(url, headers=headers, timeout=30)
            response.raise_for_status()
            
            # Determine file extension from URL / URLからファイル拡張子を決定
            file_ext = 'jpg' if 'jpg' in url.lower() else 'png'
            filename = f'lena.{file_ext}'
            
            with open(filename, 'wb') as f:
                f.write(response.content)
            
            # Load and convert image / 画像を読み込み変換
            img = cv2.imread(filename)
            if img is None:
                raise ValueError(f"Failed to load image from {filename}")
            
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            print(f"✓ Lena image downloaded successfully from source {i+1}!")
            print(f"✓ ソース {i+1} からレナ画像のダウンロードが完了しました！")
            print(f"Image shape: {img_rgb.shape}")
            print(f"Source: {url}")
            
            return img_rgb
            
        except Exception as e:
            print(f"❌ Source {i+1} failed: {e}")
            print(f"❌ ソース {i+1} が失敗: {e}")
            if i < len(urls) - 1:
                print("Trying next source... / 次のソースを試行中...")
            continue
    
    # If all sources fail, create synthetic image / 全ソースが失敗した場合、合成画像を作成
    print("All download sources failed. Creating synthetic test image.")
    print("全ダウンロードソースが失敗しました。合成テスト画像を作成中。")
    
    # Create a more realistic synthetic image instead of random noise
    # ランダムノイズの代わりにより現実的な合成画像を作成
    img = np.zeros((512, 512, 3), dtype=np.uint8)
    
    # Add some structure to make it useful for edge detection
    # エッジ検出に有用な構造を追加
    for i in range(0, 512, 64):
        img[i:i+32, :] = [128, 128, 128]  # Horizontal stripes
        img[:, i:i+32] = [64, 64, 64]    # Vertical stripes
    
    # Add a central circle for feature detection
    # 特徴検出用の中央円を追加
    center = (256, 256)
    cv2.circle(img, center, 100, (255, 255, 255), -1)
    cv2.circle(img, center, 80, (0, 0, 0), -1)
    cv2.circle(img, center, 60, (200, 200, 200), -1)
    
    print("Synthetic test image created with structured patterns.")
    print("構造化パターンを持つ合成テスト画像を作成しました。")
    
    return img
lena_img = download_lena_image()
lena_gray = cv2.cvtColor(lena_img, cv2.COLOR_RGB2GRAY)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(lena_img)
plt.title('Original Lena Image (Color)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(lena_gray, cmap='gray')
plt.title('Grayscale Lena Image')
plt.axis('off')

plt.tight_layout()
plt.show()

## 2. Smoothing Filters / 平滑化フィルタ

Smoothing filters are used to reduce noise and blur images. We'll demonstrate Gaussian and median filters.

平滑化フィルタはノイズを減らし、画像をぼかすために使用されます。ガウシアンフィルタとメディアンフィルタを実演します。

In [ ]:
# Apply different smoothing filters
gaussian_3 = cv2.GaussianBlur(lena_gray, (3, 3), 0)
gaussian_9 = cv2.GaussianBlur(lena_gray, (9, 9), 0)
gaussian_15 = cv2.GaussianBlur(lena_gray, (15, 15), 0)
median_5 = cv2.medianBlur(lena_gray, 5)
median_9 = cv2.medianBlur(lena_gray, 9)

plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.imshow(lena_gray, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(gaussian_3, cmap='gray')
plt.title('Gaussian Blur (3x3)')
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(gaussian_9, cmap='gray')
plt.title('Gaussian Blur (9x9)')
plt.axis('off')

plt.subplot(2, 3, 4)
plt.imshow(gaussian_15, cmap='gray')
plt.title('Gaussian Blur (15x15)')
plt.axis('off')

plt.subplot(2, 3, 5)
plt.imshow(median_5, cmap='gray')
plt.title('Median Filter (5x5)')
plt.axis('off')

plt.subplot(2, 3, 6)
plt.imshow(median_9, cmap='gray')
plt.title('Median Filter (9x9)')
plt.axis('off')

plt.tight_layout()
plt.show()

## 3. Edge Detection with Sobel Filters / ソーベルフィルタによるエッジ検出

Sobel filters detect edges by computing gradients in horizontal and vertical directions.

ソーベルフィルタは水平および垂直方向の勾配を計算してエッジを検出します。

In [ ]:
# Apply Sobel filters
sobel_x = cv2.Sobel(lena_gray, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(lena_gray, cv2.CV_64F, 0, 1, ksize=3)

sobel_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
sobel_direction = np.arctan2(sobel_y, sobel_x)

kernel_diag1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
kernel_diag2 = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=np.float32)

sobel_diag1 = cv2.filter2D(lena_gray.astype(np.float32), -1, kernel_diag1)
sobel_diag2 = cv2.filter2D(lena_gray.astype(np.float32), -1, kernel_diag2)

plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.imshow(lena_gray, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(np.abs(sobel_x), cmap='gray')
plt.title('Sobel X (Vertical Edges)')
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(np.abs(sobel_y), cmap='gray')
plt.title('Sobel Y (Horizontal Edges)')
plt.axis('off')

plt.subplot(2, 3, 4)
plt.imshow(sobel_magnitude, cmap='gray')
plt.title('Sobel Magnitude')
plt.axis('off')

plt.subplot(2, 3, 5)
plt.imshow(np.abs(sobel_diag1), cmap='gray')
plt.title('Diagonal Sobel 1')
plt.axis('off')

plt.subplot(2, 3, 6)
plt.imshow(np.abs(sobel_diag2), cmap='gray')
plt.title('Diagonal Sobel 2')
plt.axis('off')

plt.tight_layout()
plt.show()

## 4. SIFT Feature Detection / SIFT特徴点検出

SIFT (Scale-Invariant Feature Transform) detects distinctive keypoints that are invariant to scale and rotation.

SIFT（スケール不変特徴変換）は、スケールと回転に不変な特徴的なキーポイントを検出します。

In [ ]:
# Create SIFT detector
sift = cv2.SIFT_create()

keypoints, descriptors = sift.detectAndCompute(lena_gray, None)

img_with_keypoints = cv2.drawKeypoints(lena_img, keypoints, None, 
                                      flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

print(f"Number of SIFT keypoints detected: {len(keypoints)}")
print(f"Descriptor shape: {descriptors.shape if descriptors is not None else 'None'}")

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(lena_img)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(img_with_keypoints)
plt.title(f'SIFT Keypoints ({len(keypoints)} detected)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(lena_gray, cmap='gray')
x_coords = [kp.pt[0] for kp in keypoints]
y_coords = [kp.pt[1] for kp in keypoints]
plt.scatter(x_coords, y_coords, c='red', s=10, alpha=0.7)
plt.title('SIFT Keypoint Locations')
plt.axis('off')

plt.tight_layout()
plt.show()

## 5. Harris Corner Detection / ハリスコーナー検出

Harris corner detection finds corners by analyzing the local structure of the image.

ハリスコーナー検出は、画像の局所構造を分析してコーナーを見つけます。

In [ ]:
# Harris corner detection
harris_corners = cv2.cornerHarris(lena_gray, 2, 3, 0.04)

harris_corners = cv2.dilate(harris_corners, None)

img_harris = lena_img.copy()
img_harris[harris_corners > 0.01 * harris_corners.max()] = [255, 0, 0]

coords = feature.corner_peaks(feature.corner_harris(lena_gray), min_distance=5)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(lena_img)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(img_harris)
plt.title('Harris Corners (OpenCV)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(lena_gray, cmap='gray')
plt.plot(coords[:, 1], coords[:, 0], 'r+', markersize=8)
plt.title(f'Harris Corners (scikit-image): {len(coords)} detected')
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"OpenCV Harris corners: {np.sum(harris_corners > 0.01 * harris_corners.max())} detected")
print(f"Scikit-image Harris corners: {len(coords)} detected")

## Summary / まとめ

In this tutorial, we demonstrated various image filtering techniques:

このチュートリアルでは、様々な画像フィルタリング技術を実演しました：


1. **Image Download / 画像ダウンロード**: Successfully downloaded and processed the Lena image
   レナ画像のダウンロードと処理に成功

2. **Smoothing Filters / 平滑化フィルタ**: 
   - Gaussian blur for noise reduction / ノイズ減少のためのガウシアンぼかし
   - Median filter for salt-and-pepper noise / 塩胡椒ノイズのためのメディアンフィルタ

3. **Edge Detection / エッジ検出**: 
   - Sobel filters in X and Y directions / X・Y方向のソーベルフィルタ
   - Diagonal edge detection / 対角エッジ検出
   - Magnitude and direction computation / 大きさと方向の計算

4. **Feature Detection / 特徴検出**: 
   - SIFT keypoints for scale-invariant features / スケール不変特徴のためのSIFTキーポイント
   - Harris corner detection for structural features / 構造的特徴のためのハリスコーナー検出


These techniques form the foundation for:
- Object recognition and tracking / 物体認識と追跡
- Image registration and stitching / 画像レジストレーションと合成
- Computer vision preprocessing / コンピュータビジョン前処理
- Medical image analysis / 医用画像解析